In [ ]:
import torch
import joblib
import numpy as np
import pandas as pd
# from sklearn.decomposition import PCA
import sklearn
import sklearn.metrics

In [ ]:
storage_dir = '/n/holylabs/LABS/krajan_lab/Users/sjohnsonyu/elephantfish-talking-storage/totem_vars/'

dataset = 'real_fish_day_12ms_eods_only_seq_len_640'
mask_ratio = 0.25
seed = 11
compression_factor = 4

exp_id = f'{dataset}_mr{mask_ratio}_seed{seed}_f{compression_factor}'

codebook = joblib.load(storage_dir + f'{exp_id}_codebook.pkl')
codes = joblib.load(storage_dir + f'{exp_id}_codes.pkl')
code_ids = joblib.load(storage_dir + f'{exp_id}_code_ids.pkl')
preds = joblib.load(storage_dir + f'{exp_id}_preds.pkl')
trues = joblib.load(storage_dir + f'{exp_id}_trues.pkl')
inps = joblib.load(storage_dir + f'{exp_id}_inps.pkl')

In [ ]:
# codebook = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_codebook.pkl')
# codes = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_codes.pkl')
# code_ids = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_code_ids.pkl')
# preds = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_preds.pkl')
# trues = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_trues.pkl')

# codebook = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_seed{seed}_codebook.pkl')
# codes = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_seed{seed}_codes.pkl')
# code_ids = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_seed{seed}_code_ids.pkl')
# preds = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_seed{seed}_preds.pkl')
# trues = joblib.load(storage_dir + f'{dataset}_mr{mask_ratio}_seed{seed}_trues.pkl')

In [ ]:
concatenated_preds = torch.cat(preds, dim=0)
concatenated_trues = torch.cat(trues, dim=0)

thresholded_preds_all = np.where(concatenated_preds.cpu() > 0.5, 1, 0)
trues_all = concatenated_trues.cpu().numpy()

print('percentage of trues that are 1:', np.sum(trues_all) / len(trues_all))
print('percentage of preds that are 1:', np.sum(thresholded_preds_all) / len(thresholded_preds_all))

# precision and recall and f1 by class
metrics = sklearn.metrics.precision_recall_fscore_support(trues_all, thresholded_preds_all, average=None)

metrics_df = pd.DataFrame({
    'Precision': metrics[0],
    'Recall': metrics[1],
    'F1-Score': metrics[2],
    'Support': metrics[3]
})
print(metrics_df.round(3))
f1 = metrics_df['F1-Score'].mean()
print('Overall F1-Score:', f1.round(3))


In [ ]:
# pca = PCA(n_components=2)
# pca.fit(codes)
# codes_pca = pca.transform(codes)


In [ ]:
import sklearn
import sklearn.metrics

def get_sklearn_report(preds, trues):
    metrics = sklearn.metrics.precision_recall_fscore_support(trues, preds, average=None)
    metrics_df = pd.DataFrame({
        'Precision': metrics[0],
        'Recall': metrics[1],
        'F1-Score': metrics[2],
        'Support': metrics[3]
    })
    print(metrics_df.round(3))
    f1 = metrics_df['F1-Score'].mean()
    print('Overall F1-Score:', f1.round(3))


def run_baselines(test_data):
    test_data = test_data.flatten()
    all_0_preds = np.zeros_like(test_data)
    all_1_preds = np.ones_like(test_data)
    p_1 = np.sum(test_data) / test_data.size
    random_preds = np.random.choice([0, 1], size=test_data.size, p=[1-p_1, p_1])
    print('All 0s')
    get_sklearn_report(all_0_preds, test_data)
    print('All 1s')
    get_sklearn_report(all_1_preds, test_data)
    print('Random')
    get_sklearn_report(random_preds, test_data)

run_baselines(trues_all)

In [ ]:
# import torch

# def find_incorrect_predictions_with_context(inputs, preds, labels, window_size=5):
#     inputs_flat = inputs.view(-1)
#     preds_flat = preds.view(-1)
#     labels_flat = labels.view(-1)
#     mask_indices = (inputs_flat == -1).nonzero(as_tuple=True)[0]
#     masked_preds = preds_flat[mask_indices]
#     masked_labels = labels_flat[mask_indices]
#     incorrect_indices = (masked_preds != masked_labels).nonzero(as_tuple=True)[0]
#     incorrect_contexts = []
#     for idx in incorrect_indices:
#         print('idx:', idx)
#         if idx > 1000: continue
#         start = max(0, mask_indices[idx] - window_size)
#         end = min(inputs_flat.size(0), mask_indices[idx] + window_size + 1)
#         print('start:', start)
#         print('end:', end)
#         context_input = inputs_flat[start:end]
#         context_pred = preds_flat[start:end]
#         context_label = labels_flat[start:end]
#         incorrect_contexts.append((context_input, context_pred, context_label))
#     return incorrect_indices, incorrect_contexts

# # Example usage
# incorrect_indices, incorrect_contexts = find_incorrect_predictions_with_context(inps[0], preds[0], trues[0])
# for idx, (context_input, context_pred, context_label) in zip(incorrect_indices, incorrect_contexts):
#     print(f"Incorrect prediction at index {idx}")
#     print(f"Context input: {context_input}")
#     print(f"Context prediction: {context_pred}")
#     print(f"Context true label: {context_label}")


In [ ]:
def find_incorrect_predictions_with_context(inputs, preds, labels, window_size=5):
    inputs_np = inputs.detach().cpu().numpy().flatten()  # Convert tensor to NumPy array and flatten
    preds_np = preds.detach().cpu().numpy().flatten()
    labels_np = labels.detach().cpu().numpy().flatten()
    mask_indices = np.where(inputs_np == -1)[0]  # Find indices where the input was masked with -1
    masked_preds = preds_np[mask_indices]
    masked_labels = labels_np[mask_indices]
    incorrect_indices = np.where(masked_preds != masked_labels)[0]
    incorrect_contexts = []
    try:
        for idx in incorrect_indices:
            start = max(0, mask_indices[idx] - window_size)
            end = min(inputs_np.size, mask_indices[idx] + window_size + 1)
            context_input = inputs_np[start:end]
            context_pred = preds_np[start:end]
            context_label = labels_np[start:end]
            incorrect_contexts.append((context_input, context_pred, context_label))
    except Exception as e:
        # print('start', start)
        # print('end', end)
        # print('inputs_np.shape', inputs_np.shape)
        # print('preds_np.shape', preds_np.shape)
        print('hello')
        print(idx)
        print(mask_indices)
        print(e)
    return incorrect_indices, incorrect_contexts

# Example usage (assuming inputs, preds, and labels are PyTorch tensors)
incorrect_indices, incorrect_contexts = find_incorrect_predictions_with_context(inps[0], preds[0], trues[0])

for idx, (context_input, context_pred, context_label) in zip(incorrect_indices, incorrect_contexts):
    print(f"Incorrect prediction at index {idx}")
    print(f"Context input: {context_input}")
    print(f"Context prediction: {context_pred}")
    print(f"Context true label: {context_label}")
